In [1]:
def make_dict(sub):
    check_dict[sub] = { 
            "REST_CHECK": None,
            "FUNC_TRAIN_CHECK": None,
            "TRAIN_COUNT" :0,
            "ANAT_CHECK":None,
            "FMAP_CHECK":None
    }

In [2]:
import os, shutil, glob
import pandas as pd 

#BIDS_PATH = 
#MULTI_SESS =

#def check_subjects():
## Set input path for BIDS data and get subjects 
SUB_DIRS = glob.glob(os.path.join("/projects/niblab/bids_projects/Experiments/Bevel/BIDS", "sub-*"))
check_dict = {} ## initialize dictionary
## iterate through subjects and inspect their files
for SUBDIR in sorted(SUB_DIRS):
    SUBJECT = SUBDIR.split("/")[-1] # get subject id
    # MAKE DICTIONARY 
    make_dict(SUBJECT) # add subject id to dictionary
    #print(">>>>-------> ", SUBDIR)
    # GET RELEVANT PATHS, AND FILES 
    ## --here we set the data directory paths we want to check
    ## (func/, anat/, and fmap/ path)
    ## and then we grab all the *nii.gz files within them
    funcpath = os.path.join(SUBDIR, 'func', '*nii.gz')
    anatpath = os.path.join(SUBDIR, 'anat', '*nii.gz')
    fmappath = os.path.join(SUBDIR, 'fmap', '*nii.gz')
    FUNCS = glob.glob(funcpath)
    ANATS = glob.glob(anatpath)
    FMAPS = glob.glob(fmappath)
    # IDENTIFY FUNCS AND ANY ERRORS
    if not FUNCS:
        #print("********ERROR! NO FUNCTIONALS: ", SUBJECT)
        func_check = "FAIL"
        #######WRITE TO FILE -- QUALITY ANALYSIS REPORTING
    # IDENTIFY FUNCS
    else:
        #print("> CURRENT FUNCTION FILES: ", FUNCS)
        train_run_count=0
        rl_count = 0 
        for func_ in sorted(FUNCS):
            name = func_.split("/")[-1] 
            TASK = name.split("_")[1]
            #print(name)
            if "rest" in name:
                resting_file = name
                rest_check = "PASS"
                check_dict[SUBJECT]["REST_CHECK"] = rest_check 
            else:
                if "prob" in TASK:
                    if "run" in name:
                        TRAIN_RUN = name.split("_")[2]
                        train_run_count = train_run_count + 1
        if train_run_count == 4:
            func_check = "PASS"
        else:
            func_check = "PARTIAL FAIL"
    check_dict[SUBJECT]["TRAIN_COUNT"] = train_run_count
    check_dict[SUBJECT]["FUNC_TRAIN_CHECK"] = func_check
    
            #rint("RUN COUNT: ", run_count)
    # IDENTIFY ANATS AND ANY ERRORS
    if not ANATS:
        anat_check = "FAIL"
        #print("********ERROR! NO ANATOMICALS: ", SUBJECT)
        #######WRITE TO FILE -- QUALITY ANALYSIS REPORTING
    else:
        anat_file = str(ANATS)
        anat_check = "PASS"
    check_dict[SUBJECT]["ANAT_CHECK"] = anat_check
    # IDENTIFY FMAPS AND ANY ERRORS
    ## here I am initializing the check variables
    phase_check = "FAIL"
    mag1_check = "FAIL"
    mag2_check = "FAIL"
    if not FMAPS:
        print("********ERROR! NO FMAPS: ", SUBJECT)
        fmap_check = "FAIL"
    else:
        for file in FMAPS:
            name = file.split("/")[-1]
            IDENTIFIER = name.split("_")[1]
            if "phasediff" in IDENTIFIER:
                phase_check = "PASS"
                phase_file = file
            if "magnitude1" in IDENTIFIER:
                mag1_check = "PASS"
                mag1_file = file
            if "magnitude2" in IDENTIFIER: 
                mag2_check = "PASS"
                mag2_file = file
        #print(phase_check, mag1_check, mag2_check)
        if phase_check and mag1_check and mag2_check == "PASS":
            fmap_check = "PASS"
        elif phase_check and mag1_check and mag2_check != "PASS":
            fmap_check = "FAIL"
        else:
            fmap_check = "PARTIAL FAIL"
    check_dict[SUBJECT]["FMAP_CHECK"] = fmap_check


In [3]:
pd.DataFrame.from_dict(check_dict, orient='index')

""
